In [1]:
!pip install --upgrade google-cloud-vision

In [2]:
import os
from google.colab import files

uploaded = files.upload()

KEY_PATH = list(uploaded.keys())[0]

Saving plucky-mile-460423-g0-54d46c63149e.json to plucky-mile-460423-g0-54d46c63149e.json


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = KEY_PATH

print(f"Arquivo de credenciais carregado em: {KEY_PATH}")

Arquivo de credenciais carregado em: plucky-mile-460423-g0-54d46c63149e.json


In [4]:
uploaded_image = files.upload()
IMAGE_PATH = list(uploaded_image.keys())[0]

print(f"Imagem carregada em: {IMAGE_PATH}")

Saving novoPrint.png to novoPrint.png
Imagem carregada em: novoPrint.png


In [5]:
from google.cloud import vision

def detect_text(path):
    """Detecta texto na imagem fornecida."""
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
      print("Nenhum texto detectado na imagem.")
      return None # Retorna None se nenhum texto for encontrado

    # A primeira anotação geralmente contém todo o texto detectado na imagem
    full_text = texts[0].description

    print('Textos detectados:')
    # Imprime todas as anotações individuais para visualização
    for text in texts:
      print(f'\nTexto: "{text.description}"')
      vertices = (['({},{})'.format(v.x, v.y) for v in text.bounding_poly.vertices])
      print(f'Coordenadas dos vértices: {vertices}')

    if response.error.message:
      raise Exception(f'{response.error.message}')

    return full_text # Retorna o texto completo detectado

In [6]:
!pip install --upgrade google-cloud-translate

In [11]:
from google.cloud import translate_v3beta1 as translate
from google.colab import userdata
import os

try:
    # Tenta carregar o PROJECT_ID do Colab Secrets
    project_id = userdata.get('GOOGLE_CLOUD_PROJECT_ID')
    print("PROJECT_ID carregado do Colab Secrets.")
except Exception:
    # Se não estiver nos secrets do Colab, tenta carregar de variáveis de ambiente do sistema
    # (útil se o notebook for executado fora do Colab, mas não seguro para GitHub sem .env local)
    project_id = os.getenv('GOOGLE_CLOUD_PROJECT_ID')
    if not project_id:
        # Fallback para um placeholder ou erro se não for encontrado em nenhum lugar
        print("Atenção: GOOGLE_CLOUD_PROJECT_ID não encontrado em Colab Secrets ou variáveis de ambiente.")
        print("Por favor, configure-o em seu ambiente Colab Secrets para executar este notebook.")
        project_id = "your-google-cloud-project-id-here" # Placeholder para quem for baixar

print(f"Usando PROJECT_ID definido.")

# No caso da API Key, que você já configurou no app.yaml, mas pode estar no notebook também:
# import google.generativeai as genai
# try:
#    api_key = userdata.get('GOOGLE_API_KEY')
#    genai.configure(api_key=api_key)
#    print("API Key carregada do Colab Secrets para o Gemini.")
# except Exception:
#    api_key = os.getenv('GOOGLE_API_KEY')
#    if api_key:
#        genai.configure(api_key=api_key)
#        print("API Key carregada de variáveis de ambiente do sistema para o Gemini.")
#    else:
#        print("Atenção: GOOGLE_API_KEY não encontrada. Configure-a em seu ambiente Colab Secrets.")
#        # Ou você pode configurar com um placeholder ou levantar um erro


def detect_language(text_content, project_id):
    """Detecta o idioma de um texto.

    Args:
        text_content: O texto a ser analisado.
        project_id: O ID do seu projeto GCP.

    Returns:
        O código do idioma detectado (ex: 'en' para inglês, 'pt' para português).
    """
    if not text_content:
      print("Texto vazio, não é possível detectar o idioma.")
      return None

    client = translate.TranslationServiceClient()
    location = "global" # Para detecção e tradução, "global" geralmente é suficiente
    parent = f"projects/{project_id}/locations/{location}"

    try:
      # Passe o conteúdo dentro de um dicionário request
      response = client.detect_language(
          request={
              "parent": parent,
              "content": text_content # 'content' é uma chave válida dentro do dicionário request
          }
      )

      # A API retorna uma lista de idiomas detectados, pegamos o primeiro e mais provável
      for language in response.languages:
        print(f" Idioma detectado: {language.language_code}")
        print(f" Confiança: {language.confidence:.2f}")
        return language.language_code
      return None # Retorna None se a API não retornar idiomas

    except Exception as e:
      print(f"Erro ao detectar idioma: {e}")
      return None


def translate_text(target_language, text_content, project_id):
    """Traduz um texto para o idioma alvo.

    Args:
        target_language: O código do idioma para o qual traduzir (ex: 'pt-BR').
        text_content: O texto a ser traduzido.
        project_id: O ID do seu projeto GCP.

    Returns:
        O texto traduzido.
    """
    if not text_content:
      print("Texto vazio, não é possível traduzir.")
      return None

    client = translate.TranslationServiceClient()
    location = "global"
    parent = f"projects/{project_id}/locations/{location}"

    try:
      response = client.translate_text(
          request={
              "parent": parent,
              "contents": [text_content],
              "mime_type": "text/plain", # Ou "text/html" se o conteúdo for HTML
              "target_language_code": target_language,
          }
      )

      for translation in response.translations:
        return translation.translated_text
      return None # Retorna None se a API não retornar traduções

    except Exception as e:
      print(f"Erro ao traduzir texto: {e}")
      return None

# --- Exemplo de uso integrado ---

# Você precisará do ID do seu projeto GCP.

# Chame a função detect_text para obter o texto da imagem
# Certifique-se que IMAGE_PATH está definida a partir de uma célula anterior.
extracted_text = detect_text(IMAGE_PATH)


if extracted_text:
  print(f"\n--- Processando o texto extraído: '{extracted_text}' ---")

  detected_lang = detect_language(extracted_text, YOUR_PROJECT_ID)

  if detected_lang and detected_lang != 'pt':
    print(f"\nIdioma detectado: '{detected_lang}'. Traduzindo para o português...")
    translated_text = translate_text('pt', extracted_text, YOUR_PROJECT_ID)
    if translated_text:
      print(f"Texto traduzido: '{translated_text}'")
    else:
      print("Falha ao traduzir o texto.")
  else:
    print("\nO texto já está em português ou não foi possível detectar o idioma.")
    print(f"Texto original (ou em português): '{extracted_text}'")
else:
  print("\nNão foi possível extrair texto da imagem para processamento.")


PROJECT_ID carregado do Colab Secrets.
Usando PROJECT_ID definido.
Textos detectados:

Texto: "REMOVAL
1.
Remove left and right boots (WP 1056 00, Yaw Control Pedal Boots).
2. Remove floor panel around pedal assembly (WP 0237 00, Cockpit Floor).
Yaw Control Pedal Boots
Pedal adjuster should be in retracted position prior to removal.
NOTE
3.
Remove screws and cover over directional control rod.
4.
Remove bolt, washer, and nut securing directional control rod to pedal adjuster arm (Figure 1, Sheet 2).
5. Remove bolt, washer, and nut securing pedal directional control rods to pedal adjuster links.
6. Remove bolt and nut securing release cable to pedal adjuster assembly (Figure 1, Sheet 1).
7.
8.
Remove bolt and washer securing upper and lower support bearings to support flange (Figure 1, Sheet 2, Detail B).
Remove adjuster and support bearings from between airframe supports. Remove support bearing from adjuster.
END OF TASK
TEST AND INSPECTION
1. Check upper support bearing for installati

In [12]:
!pip install -q -U google-generativeai

In [13]:
import os
import google.generativeai as genai

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

try:
    # Tenta obter a chave de ambiente, que é a forma mais segura para produção
    # ou se você configurou o Secret do Colab e o Colab populou a variável de ambiente.
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

    # Se não encontrou na variável de ambiente, tenta pegar dos secrets do Colab
    if not GOOGLE_API_KEY:
        from google.colab import userdata
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    if not GOOGLE_API_KEY:
        raise ValueError("Chave GOOGLE_API_KEY não encontrada. Por favor, configure-a.")

    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configurada com sucesso!")

except Exception as e:
    print(f"Erro ao configurar a Gemini API: {e}")
    print("Por favor, certifique-se de que sua chave GOOGLE_API_KEY está configurada corretamente.")

Gemini API configurada com sucesso!


In [14]:
import google.generativeai as genai

def simplify_maintenance_text(translated_text):
    """
    Simplifica e resume um texto de instrução de manutenção usando a Gemini API.

    Args:
        translated_text: O texto original da instrução de manutenção (em português).

    Returns:
        O texto simplificado e resumido.
    """
    try:
        # Inicializa o modelo Gemini
        # Podemos escolher entre modelos como 'gemini-pro' para texto
        # ou 'gemini-pro-vision' para texto e imagem (mas neste caso, já extraímos o texto)
        model = genai.GenerativeModel('gemini-2.0-flash')

        # Cria um prompt (instrução) claro para o modelo
        prompt = f"""
        Você é um assistente especializado em manutenção de aeronaves, com a tarefa de simplificar
        instruções técnicas. Recebi a seguinte instrução de manutenção de um manual:

        "{translated_text}"

        Por favor, reescreva esta instrução de forma mais simples, clara e sucinta,
        mantendo apenas as informações essenciais para um técnico realizar a tarefa.
        Use linguagem direta e evite jargões desnecessários, se possível.
        """

        # Gera a resposta do modelo
        response = model.generate_content(prompt)

        # Retorna o texto gerado pela IA
        if response and response.candidates and response.candidates[0].content.parts:
            return response.candidates[0].content.parts[0].text
        else:
            return "Não foi possível gerar uma explicação simplificada."

    except Exception as e:
        print(f"Erro ao usar a Gemini API: {e}")
        return "Ocorreu um erro ao simplificar a instrução. Por favor, tente novamente."

# --- Exemplo de uso ---

# Vamos simular um texto de manual já traduzido para o português (se necessário)
maintenance_instruction = {translated_text}

print(f"--- Instrução Original Traduzida ---")
print(translated_text)

print(f"\n--- Instrução Simplificada pela IA ---")
simplified_text = simplify_maintenance_text(translated_text)
print(simplified_text)

--- Instrução Original Traduzida ---
REMOÇÃO
1.
Remova as capas esquerda e direita (WP 1056 00, Capas do Pedal de Controle de Guinada).
2. Remova o painel do assoalho ao redor do conjunto do pedal (WP 0237 00, Assoalho da Cabine).
Capas do Pedal de Controle de Guinada
O ajustador do pedal deve estar na posição retraída antes da remoção.
NOTA
3.
Remova os parafusos e a tampa sobre a haste de controle direcional.
4.
Remova o parafuso, a arruela e a porca que prendem a haste de controle direcional ao braço do ajustador do pedal (Figura 1, Folha 2).
5. Remova o parafuso, a arruela e a porca que prendem as hastes de controle direcional do pedal às conexões do ajustador do pedal.
6. Remova o parafuso e a porca que prendem o cabo de liberação ao conjunto do ajustador do pedal (Figura 1, Folha 1).
7.
8.
Remova o parafuso e a arruela que prendem os rolamentos de suporte superior e inferior ao flange de suporte (Figura 1, Folha 2, Detalhe B).
Remova o ajustador e os rolamentos de suporte entre o